In [2]:
%pip install mido

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import mido

In [ ]:
notes = []

In [2]:
def midi_to_note_name(midi_number):
    notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
    # Calculate the octave
    octave = (midi_number // 12) - 1  # MIDI note 0 is C-1, so adjust for C0 being octave 0
    
    # Calculate the note index within the octave
    note_index = midi_number % 12
    
    # Combine the note name and octave
    note_name = notes[note_index] #+ str(octave)
    return note_name

In [ ]:
mid = mido.MidiFile('Untitled (3).mid')
print('Track {}: {}'.format(0, mid.tracks[0].name))
with open('output.txt', 'w') as f:
    for msg in mid.tracks[0]:
        if(msg.type == 'set_tempo'):
            tempo = int(mido.tempo2bpm(msg.tempo))
            f.write("b" + str(tempo) + '\n')
        elif(msg.type == 'note_on'):
            print(midi_to_note_name(msg.note))
        print(str(msg))

Track 0: Electric Piano
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=468750, time=0)
MetaMessage('track_name', name='Electric Piano', time=0)
program_change channel=0 program=0 time=0
G
note_on channel=0 note=55 velocity=50 time=0
note_off channel=0 note=55 velocity=0 time=384
A
note_on channel=0 note=57 velocity=50 time=0
note_off channel=0 note=57 velocity=0 time=384
B
note_on channel=0 note=59 velocity=50 time=0
note_off channel=0 note=59 velocity=0 time=384
C
note_on channel=0 note=60 velocity=50 time=0
note_off channel=0 note=60 velocity=0 time=384
D
note_on channel=0 note=62 velocity=50 time=0
note_off channel=0 note=62 velocity=0 time=96
E
note_on channel=0 note=64 velocity=50 time=0
note_off channel=0 note=64 velocity=0 time=96
F#
note_on channel=0 note=66 velocity=50 time=0
note_off channel=0 note=66 velocity=0 time=96
G
note_on channel=0 note=67 velocity=50 time=0
note_off 

In [16]:
import mido
from collections import defaultdict

# Assumes midi_to_note_name(note_number) exists in the environment

def export_midi_to_custom_text(midi_filename, output_filename):
    mid = mido.MidiFile(midi_filename)
    track = mid.tracks[0]
    ticks_per_beat = mid.ticks_per_beat

    # Gather note on/off events with absolute tick times
    abs_tick = 0
    note_starts = defaultdict(list)
    notes_intervals = []

    for msg in track:
        abs_tick += msg.time
        if msg.type == 'note_on' and msg.velocity > 0:
            note_starts[msg.note].append(abs_tick)
        elif (msg.type == 'note_off') or (msg.type == 'note_on' and msg.velocity == 0):
            if note_starts.get(msg.note):
                start_tick = note_starts[msg.note].pop(0)
                notes_intervals.append((msg.note, start_tick, abs_tick))
        elif msg.type == 'set_tempo':
            output_text = f'b{int(mido.tempo2bpm(msg.tempo))}\n'

    last_tick = abs_tick
    for note, starts in note_starts.items():
        for st in starts:
            notes_intervals.append((note, st, last_tick))

    if not notes_intervals:
        with open(output_filename, 'w') as f:
            f.write('')
        print('No notes found; wrote empty file.')
        return

    # Convert ticks -> beats
    intervals_in_beats = []
    for note, st, en in notes_intervals:
        start_beat = st / ticks_per_beat
        end_beat = en / ticks_per_beat
        intervals_in_beats.append((note, start_beat, end_beat))

    max_beat = max(end for _, _, end in intervals_in_beats)
    total_beats = int(max_beat + 0.9999)

    beat_outputs = []
    for beat_idx in range(total_beats):
        start = beat_idx
        end = beat_idx + 1
        sounding = []

        for note, st, en in intervals_in_beats:
            if not (en <= start or st >= end):
                overlap_start = max(st, start)
                overlap_end = min(en, end)
                overlap_duration = overlap_end - overlap_start
                remaining_from_beat_start = en - max(st, start)
                note_name = midi_to_note_name(note)
                sounding.append((note_name, overlap_start, overlap_end,
                                 overlap_duration, remaining_from_beat_start, st))

        if not sounding:
            beat_outputs.append('-')
        else:
            # sort by start then original global start then name
            sounding.sort(key=lambda x: (x[1], x[5], x[0]))

            parts = []
            prev_overlap_end = None
            eps = 1e-9  # very small tolerance

            for note_name, overlap_start, overlap_end, overlap_duration, remaining, global_start in sounding:
                # pick marker based on remaining duration
                marker = ''
                if remaining <= 0.26:
                    marker = '/'
                elif abs(remaining - (1.0/3.0)) <= 0.06:
                    marker = '*'
                elif remaining <= 0.51:
                    marker = '\\'
                else:
                    marker = ''

                token = f"{note_name}{marker}"

                if prev_overlap_end is None:
                    parts.append(token)
                else:
                    # IMPORTANT CHANGE:
                    # treat notes as overlapping (chord) ONLY when overlap_start < prev_overlap_end - eps
                    # if overlap_start == prev_overlap_end (touching) => sequential => insert space
                    if overlap_start < prev_overlap_end - eps:
                        # truly overlapping: merge into same chord (no space)
                        parts[-1] = parts[-1] + token
                    else:
                        # sequential or touching: separate with a space
                        parts.append(token)

                # update prev_overlap_end to the furthest end so following notes are compared against it
                prev_overlap_end = max(prev_overlap_end or 0.0, overlap_end)

            # join sequential groups with spaces (simultaneous notes are already concatenated)
            beat_str = ' '.join(parts)
            beat_outputs.append(beat_str)

    output_text += ' '.join(beat_outputs)
    with open(output_filename, 'w') as f:
        f.write(output_text)

    print(f'Exported {total_beats} beats to {output_filename}')


In [17]:
export_midi_to_custom_text('Song.mid', 'output.txt')

Exported 30 beats to output.txt


In [44]:
f.close()

In [31]:
print(book)

b110
b110 
b110 
b110 
hi
